In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
page = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vTNXgxlcihtmzIbzHDsQH5CXI6aSXfsZzWB7E8IC0sf4CaMsgP_p4DRSwx6TtoektFRCL3wO5m64JLB/pubhtml#")
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
allTables = soup.find_all("table")
summaryTable = allTables[0]
weeklyTables = allTables[1:]
weeklyTables = weeklyTables[::-1]

In [4]:
columns = ["week", "away", "home", "site", "spread", "total", "sp_spread", "sp_total", "sp_winProb", "f_spread", "f_winProb"]
df = pd.DataFrame(columns=columns)

In [5]:
week = 0
for table in weeklyTables:
    for tr in table.find_all("tr")[2:]:
        row = {}
        datarow = []
        
        for td in tr.children:
            #print(len(datarow), td.get_text())
            datarow.append(td.get_text())
        
        row["week"] = week
        
        mo = re.fullmatch(r"(.+) ((at)|(vs\.)) (.+)", datarow[3])
        row["away"] = re.sub(r"No. \d\d? ", r"", mo.group(1))
        row["home"] = re.sub(r"No. \d\d? ", r"", mo.group(5))
        
        if "vs." in datarow[3]:
            row["site"] = "neutral"
        else:
            row["site"] = "home"
        
        if datarow[5]:
            mo = re.fullmatch(r"(.+) (-.+)", datarow[5])
            if mo.group(1) == row["home"]:
                row["spread"] = mo.group(2)
            elif mo.group(1) == row["away"]:
                row["spread"] = "+" + mo.group(2)[1:]
            else:
                assert(False)
        else:
            row["spread"] = ""
        
        row["total"] = datarow[6]
        
        mo = re.fullmatch(r"(.+) by (\d\d?\.\d) \((\d\d?\.\d)-(\d\d?\.\d)\)", datarow[7])
        if mo.group(1) == row["home"]:
            row["sp_spread"] = "-" + mo.group(2)
        elif mo.group(1) == row["away"]:
            row["sp_spread"] = "+" + mo.group(2)
        else:
            assert(False)
        row["sp_total"] = str(round(float(mo.group(3)) + float(mo.group(4)), 1))
        
        row["sp_winProb"] = datarow[8]
        
        mo = re.fullmatch(r"(.+) by (\d\d?\.\d)", datarow[11])
        if mo.group(1) == row["home"]:
            row["f_spread"] = "-" + mo.group(2)
        elif mo.group(1) == row["away"]:
            row["f_spread"] = "+" + mo.group(2)
        else:
            assert(False)
        
        row["f_winProb"] = datarow[12]

        #print(row)
        #print()

        df = df.append(row, ignore_index=True)
    week += 1
    #print()

df = df[df.spread != ""]
df

,week,away,home,site,spread,total,sp_spread,sp_total,sp_winProb,f_spread,f_winProb
2,0,Hawaii,Colorado State,home,-17,58,-19.1,60.7,86.5%,-8.6,69.0%
3,0,Wyoming,New Mexico State,home,+5,46,+3.8,45.2,58.8%,+6.5,64.7%
6,1,New Mexico State,Minnesota,home,-21.5,46.5,-8.6,48.6,69.0%,-16.3,82.8%
7,1,Central Florida,Connecticut,home,+24,71.5,+27.0,74.0,94.1%,+28.3,94.9%
8,1,Northwestern,Purdue,home,-2,53,+1.0,54.7,52.3%,+4.5,60.2%
13,1,Wake Forest,Tulane,home,+6,57.5,+9.5,64.7,70.9%,+14.8,80.5%
16,1,Syracuse,Western Michigan,home,+5,64.5,-0.4,62.0,51.0%,-1.3,52.9%
18,1,Army,Duke,home,-12,45,-11.7,55.1,75.0%,-20.0,87.6%
19,1,Utah State,Michigan State,home,-23.5,50.5,-20.0,48.4,87.6%,-23.4,91.2%
21,1,San Diego State,Stanford,home,-14,48.5,-12.0,60.1,75.5%,-14.7,80.2%


In [6]:
df.to_csv("spHistoryThrough11.tsv", sep="\t", index=False)